In [1]:
import pandas as pd
import numpy as np
import ipywidgets as w
from IPython.display import HTML
import bqplot as bq
import copy
import traitlets as tr

In [24]:

MXF_ICON = w.Image(
    value=open("mxf-open-circle-icon.png", "rb").read(),
    format='png',
    width=60,
    height=40,
)
MXF_ICON

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x02X\x00\x00\x02X\x08\x06\x00\x00\x00\xbef\x98\xdc\x…

In [27]:
MAP_COLORS = {'Heating': '#c22324',
 'Domestic Hot Water': '#ff7f0e',
 'Heating & Domestic Hot Water': '#eb4f18',
 'Cooling': '#1f77b4',
 'Auxiliary': '#9467bd',
 'Lighting': '#17becf',
 'Small Power': '#d3d3d3',
 'IT/Servers': '#8c564b',
 'PV Generation': '#bcbd22',
 'Catering': '#2ca02c',
 'Lifts': '#e377c2',
 'Other': '#000000',
 'Unknown': '#7C7270'}

TITLE = "<h1>NHS Archetypes - Decarbonisation Pathways Analysis and Visualisation</h1>"
DESCRIPTION = """<ul>
  <li>This app is a dummy demo, illustrating the type of front-end that can be expected for visualising different decarbonisation pathways</li>
  <li>It runs on <code>python</code> code in the browser and can be deployed as a static HTML website</li>
  <li>This is an illustrative example but the calculation would be developed to consider the different archetypes within the NHS portfolio</li>
  <li>Choose from the possible design interventions to understand the incremental savings that are available (the numbers are made up)</li>
</ul> 
"""

SELECT_DESCRIPTION = "<b>Cntrl+click to Select multiple possible the design interventions:</b>"
Y_AXIS_LABEL = "Building Carbon Content (tonnes of CO2/yr)"
FIGURE_TITLE = "Annual Energy Consumption"

class EnergyConsumption(w.VBox):     
    total_saving = tr.Float()

    @tr.observe("total_saving")
    def obs_total_saving(self, on_change):
        self.html_total_saving.value = f"<b>Total Saving</b> = {self.total_saving}%"
        
    def __init__(self, data: dict[str, np.array], savings):
        if "existing building" not in data:
            raise ValueError("`existing building` data missing.")
        self.data = data
        self.savings = savings
        # options = [k for k in data.keys() if k != "existing building"]
        self.html_total_saving = w.HTML()
        self.hbx_title = w.HBox([MXF_ICON, w.HTML(TITLE)])
        self.select = w.SelectMultiple(options=list(self.savings.keys()), layout={"height": "150px", "width": "400px"})
        self.vbx_select = w.VBox([w.HTML(SELECT_DESCRIPTION), self.select])
        self.hbx_topbar = w.HBox([w.VBox([self.hbx_title, self.html_total_saving, w.HTML(DESCRIPTION)]), self.vbx_select], layout={"justify_content":"space-between"})
        self.xs = bq.OrdinalScale()  # ordinal scale to represent categorical data
        self.ys = bq.LinearScale()
        self.xax = bq.Axis(scale=self.xs, grid_lines="none")  # no grid lines needed for x
        self.yax = bq.Axis(
            scale=self.ys, orientation="vertical", label=Y_AXIS_LABEL
        )
        self.bar = bq.Bars(
            x=np.array(list(self.data.keys())),
            y=np.transpose(self.calc_y),
            scales={"x": self.xs, "y": self.ys},
            type="stacked",
            colors=colors,
            labels=labels,
            display_legend=True,
            padding=0.4,
        )
        self.fig = bq.Figure(title=FIGURE_TITLE, axes=[self.xax, self.yax], legend_location="top-right")#
        super().__init__([self.hbx_topbar, self.fig])
        self._init_controls()
        self._set_chart()

    def _init_controls(self):
        self.select.observe(self._set_chart)

    def _get_y(self, data: dict[str, np.array]) -> np.array:
        data_copy = copy.deepcopy(data)
        baseline = data_copy.pop("existing building")
        arrays = np.array(list(data_copy.values()))
        input_y = np.array([baseline])
        for idx, array in enumerate(arrays):
            diff = input_y[-1] - arrays[idx]
            input_y = np.vstack([input_y, diff])
        return input_y

    @property
    def filtered_data(self):
        return {"existing building": self.data["existing building"]} | {k: v for k, v in self.data.items() if k in self.select.value}
    
    @property
    def calc_y(self):
        return self._get_y(self.filtered_data)
        
    def _set_chart(self, onchange=None):
        self.bar = 
        self.xs.domain = ["existing building"] + list(self.select.value)
        self.fig.marks = [self.bar]
        self._update_saving()

    def _update_saving(self):
        y = self.calc_y
        self.total_saving = np.round((1 - (y[-1].sum() / y[0].sum())) * 100, 0)
        
labels = ['Heating', 'Domestic Hot Water'] # , 'Auxiliary', 'Lighting', 'Small Power', 'PV Generation'
colors = [MAP_COLORS[x] for x in labels]
data = {
    "existing building": np.array([60, 20]),
    "replace glazing": np.array([10, 0]),
    "improve airtightness": np.array([5, 0]),
    "add efficient heat source": np.array([5, 12]),
    "improve controls": np.array([0, 0]),
    "update lighting to LED": np.array([0, 0]),
    "add PV panels": np.array([0, 0]),
}
savings = {
    "replace glazing": 0.1,
    "improve air tightness": 0.1,
    "add roof insulation": 0.1,
    "replace gas heating for air source heat pump": 0.7
}

chart = EnergyConsumption(data)
display(chart)

EnergyConsumption(children=(HBox(children=(VBox(children=(HBox(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x…

In [47]:
data = {
    "existing building": np.array([60, 20]),
    # "replace glazing": np.array([10, 0]),
    # "improve airtightness": np.array([5, 0]),
    # "add efficient heat source": np.array([5, 12]),
    # "improve controls": np.array([0, 0]),
    # "update lighting to LED": np.array([0, 0]),
    # "add PV panels": np.array([0, 0]),
}
savings = {
    "replace glazing": 0.1,
    "improve air tightness": 0.1,
    "add roof insulation": 0.1,
    "replace gas heating for air source heat pump": 0.7
}

selected = ["replace glazing", "improve air tightness", "replace gas heating for air source heat pump"]

def calc_savings(data, selected):
    keys = list(data.keys())
    for n, x in enumerate(selected):
        data[x] = data[keys[n]] - (data[keys[n]] * savings[x])
        keys = list(data.keys())
    return data

calc_savings(data, selected)

{'existing building': array([60, 20]),
 'replace glazing': array([54., 18.]),
 'improve air tightness': array([48.6, 16.2]),
 'replace gas heating for air source heat pump': array([14.58,  4.86])}

In [67]:
def create_plot(x, y):
    # create two vectors x and y to plot a bar chart
    x = list("ABCDE")
    y = np.random.rand(5)
    colors = ["red", "green"]
    # 1. Create the scales
    xs = bq.OrdinalScale()  # ordinal scale to represent categorical data
    ys = bq.LinearScale()
    
    # 2. Create the axes for x and y
    xax = bq.Axis(scale=xs, label="X", grid_lines="none")  # no grid lines needed for x
    yax = bq.Axis(
        scale=ys, orientation="vertical", label="Y", tick_format=".0%"
    )  # note the use of tick_format to format ticks
    
    # 3. Create a Bars mark by passing in the scales
    bar = bq.Bars(x=x, y=y, scales={"x": xs, "y": ys}, padding=0.5, colors=colors)
    
    # 4. Create a Figure object by assembling marks and axes
    fig = bq.Figure(marks=[bar], axes=[xax, yax], title="Bar Chart")
    return fig, bar

def update_plot(x, y, bar):
    # use the hold_sync() context manager
    with bar.hold_sync():
        bar.x = x
        bar.y = y
    
x = list("ABCDE")
y = np.random.rand(5)
fig, bar = create_plot(x, y)
fig

Figure(axes=[Axis(grid_lines='none', label='X', scale=OrdinalScale()), Axis(label='Y', orientation='vertical',…

In [74]:
x = random.choices(string.ascii_letters,k=5)
y = np.random.rand(5)
update_plot(x, y, bar)



In [69]:
import string
import random

# using random.choices() generating random strings
res = ''.join(random.choices(string.ascii_letters,k=5))
                             

print(str(res))


QtSKy


array([0.15853944, 0.26520711, 0.44555846, 0.384998  , 0.16493682])